In [15]:
import pandas as pd
import numpy as np
from scipy import stats
from tqdm.notebook import tqdm_notebook as tqdm
from statsmodels.sandbox.stats.multicomp import multipletests

In [5]:
data_auc = pd.read_csv("cmf.txt", header=0, sep="\t", index_col=0)
data_auc.head()

,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
adult (sample),0.763,0.768,0.771,0.798
breast cancer,0.599,0.591,0.590,0.569
breast cancer wisconsin,0.954,0.971,0.968,0.967
cmc,0.628,0.661,0.654,0.657
ionosphere,0.882,0.888,0.886,0.898


In [18]:
data_corr = []
for i, lhr in tqdm(enumerate(data_auc.columns), total=data_auc.shape[1]):
    for j, rhr in enumerate(data_auc.columns):
        if i >= j:
            continue
        corr, p = stats.wilcoxon(data_auc[lhr], data_auc[rhr])
        data_corr.append([lhr, rhr, corr, p])

  0%|          | 0/4 [00:00<?, ?it/s]

/Users/molozey/PycharmProjects/Coursera_Machine_2/venv/lib/python3.9/site-packages/scipy/stats/morestats.py:2967: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [22]:
corr_pd = pd.DataFrame(data_corr, columns=['CLASS_1', 'CLASS_2', 'STAT', 'P_VALUE']).sort_values(by='P_VALUE')
corr_pd.head()

,CLASS_1,CLASS_2,STAT,P_VALUE
0,C4.5,C4.5+m,6.5,0.010757
2,C4.5,C4.5+m+cf,11.0,0.015874
5,C4.5+cf,C4.5+m+cf,10.5,0.025314
3,C4.5+m,C4.5+cf,18.0,0.054329
4,C4.5+m,C4.5+m+cf,22.0,0.327826


In [24]:
#   Поправка Холма
reject, p_corrected, a1, a2 = multipletests(corr_pd.P_VALUE, alpha=.05, method="holm")
corr_pd["P_CORRECTED"] = p_corrected
corr_pd["REJECT"] = reject

In [25]:
corr_pd.head()

,CLASS_1,CLASS_2,STAT,P_VALUE,P_CORRECTED,REJECT
0,C4.5,C4.5+m,6.5,0.010757,0.064543,False
2,C4.5,C4.5+m+cf,11.0,0.015874,0.079372,False
5,C4.5+cf,C4.5+m+cf,10.5,0.025314,0.101254,False
3,C4.5+m,C4.5+cf,18.0,0.054329,0.162986,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.655651,False


In [26]:
#   Бенджамини
reject, p_corrected, a1, a2 = multipletests(corr_pd.P_VALUE, alpha=.05, method="fdr_bh")
corr_pd["P_CORRECTED"] = p_corrected
corr_pd["REJECT"] = reject

In [27]:
corr_pd.head()

,CLASS_1,CLASS_2,STAT,P_VALUE,P_CORRECTED,REJECT
0,C4.5,C4.5+m,6.5,0.010757,0.047623,True
2,C4.5,C4.5+m+cf,11.0,0.015874,0.047623,True
5,C4.5+cf,C4.5+m+cf,10.5,0.025314,0.050627,False
3,C4.5+m,C4.5+cf,18.0,0.054329,0.081493,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.393391,False
